# Indexation de table

## Pourquoi indexer une table ?

Jusqu'ici, on a vu que __pour extraire de l'information d'une table, il fallait toujours parcourir l'ensemble de cette table__. On peut aisément comprendre que ce coût est très important lorsqu'il s'agit de traiter des tables de grande taille.

Il devient alors intéressant de créer de nouvelles structures pour accélérer des recherches type. Comme on l'a vu (et on le détaillera un peu plus tard), les dictionnaires ont des qualités intrinsèques qui permettent de trouver une valeur efficacement, par clé plutôt que par indice. 

Il suffit donc de créer un nouveau dictionnaire, en choisissant les bonnes clés. Celles qui seront à l'origine des recherches futures.

Ce principe s'appelle l'__indexation__ : on crée un __dictionnaire, avec des clés judicieusement choisies, pour améliorer les performances de recherche dans une table__.

Dans le cas d'une table, __l'indexation consite donc souvent à créer un dictionnaire de dictionnaires__.

Reprenons notre exemple de table de naissances...

In [ ]:
import csv

with open("244400404_prenoms-enfants-nes-nantes.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    table_naissances = [{key : value for key, value in element.items()} for element in reader]

print(table_naissances)

Quel type de recherche pourrait être la plus fréquente sur cette table ?

Il est raisonnable de penser que les __recherches par prénom__ seront très utiles à optimiser car potentiellement les plus fréquentes.

## Comment indexer une table ?

Que pourrait-on chercher comme information tirée de cette table lorsqu'on fait une __recherche sur un prénom__ ?

Là encore, c'est subjectif, mais le plus probable est que vous souhaitiez __connaître le nombre de naissances, pour telle ou telle année__. Sur un __site web dédié aux prénoms__ par exemple.

### Indexation par prénom

Nous allons créer un index favorisant une recherche par prénom.

In [ ]:
index_par_prenom = {}
for element in table_naissances:
    if element['Prénom'] in index_par_prenom:
        index_par_prenom[element['Prénom']].update({element['Année de naissance'] : element['Occurence']})
    else:
        index_par_prenom[element['Prénom']] = {element['Année de naissance'] : element['Occurence']}

print(index_par_prenom)

En passant, cet index nous permet de savoir très facilement __combien de prénoms différents__ (dont l'occurrence annuelle est supérieure ou égale à 5) ont été donnés pendant cette période :

In [ ]:
len(index_par_prenom)

La recherche sur un prénom se fait maintenant de façon __très simple et très rapide__.

In [ ]:
index_par_prenom['DAVID']

> __Remarque :__ la tentation de trier ce résultat par date est forte mais...

In [ ]:
index_par_prenom['DAVID'].sort()

__Un dictionnaire n'étant pas ordonnée, on ne peut pas le trier !__

Qu'à cela ne tienne, il suffit de créer une liste à partir du dictionnaire.

In [ ]:
liste_david = list(index_par_prenom['DAVID'].items())
liste_david.sort()

print(liste_david)

Cet index nous permet bien un traitement aisé et efficace lors d'une recherche par prénom. Mais qu'en est-il d'une __recherche par année__ ?

### Indexation par année

Si on veut connaître les prénoms les plus donnés telle ou telle année, cet index serait particulièrement inadapté. Il faut donc créer un nouvel index dédié à ce type de recherche.

A vous de jouer...

In [ ]:
index_par_annee = {}
for element in table_naissances:
    if element['Année de naissance'] in index_par_annee:
        index_par_annee[element['Année de naissance']].update({element['Prénom'] : element['Occurence']})
    else:
        index_par_annee[element['Année de naissance']] = {element['Prénom'] : element['Occurence']}

print(index_par_annee)

Trouver les __prénoms les plus populaires en 2008__ :

In [ ]:
print(index_par_annee['2008'])

Trouver les __10 prénoms__ les plus populaires en 2019 :

In [ ]:
prenoms_2019 = list(index_par_annee['2019'].items())

prenoms_2019.sort(key=lambda x: float(x[1]), reverse=True)

print(prenoms_2019[:11])

### Indexation pour une recherche par sexe (Approfondissement)

Même principe que précédemment : créer un index pour anticiper une __recherche par année uniquement sur les prénoms de fille__.

In [ ]:
index_par_annee_fille = {}
for element in table_naissances:
    if element['Sexe'] == 'FILLE':
        if element['Année de naissance'] in index_par_annee_fille:
            index_par_annee_fille[element['Année de naissance']].update({element['Prénom'] : element['Occurence']})
        else:
            index_par_annee_fille[element['Année de naissance']] = {element['Prénom'] : element['Occurence']}

print(index_par_annee_fille)

Trouver les __10 prénoms de fille__ les plus populaires en 2004 :

In [ ]:
prenoms_fille_2004 = list(index_par_annee_fille['2004'].items())

prenoms_fille_2004.sort(key=lambda x: float(x[1]), reverse=True)

print(prenoms_fille_2004[:11])

Créer un index pour anticiper une __recherche par sexe et par année__. C'est à dire, un index permettant de faire ce type de recherche :

`index_par_annee_et_sexe['GARCON']['2003']`

In [ ]:
index_par_annee_et_sexe = {}
for element in table_naissances:
    if element['Sexe'] == 'FILLE':
        if 'FILLE' in index_par_annee_et_sexe:
            if element['Année de naissance'] in index_par_annee_et_sexe['FILLE']:
                index_par_annee_et_sexe['FILLE'][element['Année de naissance']].update({element['Prénom'] : element['Occurence']})
            else:
                index_par_annee_et_sexe['FILLE'][element['Année de naissance']] = {element['Prénom'] : element['Occurence']}
        else:
            index_par_annee_et_sexe['FILLE'] = {element['Année de naissance'] : {element['Prénom'] : element['Occurence']}}
    else:
        if 'GARCON' in index_par_annee_et_sexe:
            if element['Année de naissance'] in index_par_annee_et_sexe['GARCON']:
                index_par_annee_et_sexe['GARCON'][element['Année de naissance']].update({element['Prénom'] : element['Occurence']})
            else:
                index_par_annee_et_sexe['GARCON'][element['Année de naissance']] = {element['Prénom'] : element['Occurence']}
        else:
            index_par_annee_et_sexe['GARCON'] = {element['Année de naissance'] : {element['Prénom'] : element['Occurence']}}
print(index_par_annee_et_sexe)

In [ ]:
index_par_annee_et_sexe['GARCON']['2003']

Trouver les __10 prénoms de garçon les plus populaires en 2003__ :

In [ ]:
prenoms_garcon_2003 = list(index_par_annee_et_sexe['GARCON']['2003'].items())

prenoms_garcon_2003.sort(key=lambda x: float(x[1]), reverse=True)

print(prenoms_garcon_2003[:11])

---
[![Licence CC BY NC SA](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png "licence Creative Commons CC BY-NC-SA")](http://creativecommons.org/licenses/by-nc-sa/3.0/fr/)
<p style="text-align: center;">Auteur : David Landry, Lycée Clemenceau - Nantes</p>